# Previsão de Vendas das Lojas Rossmann

### Este projeto foi orientado pela Comunidade DS, utilizando os dados disponíveis no Kaggle da Rede de Lojas Rossmann. 

### Importação de Bibliotecas

In [ ]:
import math
import pandas as pd
import random
import inflection
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import xgboost as xgb
import random
import pickle
import warnings


from scipy import stats as ss
from IPython.display import Image
from IPython.core.display import HTML


from boruta import BorutaPy
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso

warnings.filterwarnings('ignore')

In [ ]:
def cross_validation(x_training, kfold, model_name, model, verbose=False):
    mae_list = []
    mape_list = []
    rmse_list = []
    
    for k in reversed(range(1, kfold +1 )):
        if verbose:
            print(f'\nKFold Number: {k}')
            
        # start and end date for validation
        validation_start_date = x_training['date'].max() - datetime.timedelta(days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta(days=(k-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) &
                                (x_training['date'] <= validation_end_date)]

        # training and validation dataset
        # training 
        xtraining = training.drop(['date', 'sales'], axis=1)
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop(['date', 'sales'], axis=1)
        yvalidation = validation['sales']

        # model
        m = model.fit(xtraining, ytraining)

        # prediction
        yhat = m.predict(xvalidation)

        # performance
        m_result = ml_error(model_name, np.expm1(yvalidation), np.expm1(yhat))

        # store performance of each kfold iteration
        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])

        # dataframe
        return pd.DataFrame({'Model Name': model_name,
                        'MAE CV': np.round(np.mean(mae_list),2 ).astype(str) + ' +/- ' + np.round(np.std(mae_list),2 ).astype(str),
                        'MAPE CV': np.round(np.mean(mape_list),2 ).astype(str) + ' +/- ' + np.round(np.std(mape_list),2 ).astype(str),
                        'RMSE CV': np.round(np.mean(rmse_list),2 ).astype(str) + ' +/- ' + np.round(np.std(rmse_list),2 ).astype(str)}, index=[0])

def mean_absolute_percentage_error(y, yhat):
    return np.mean(np.abs( ( y - yhat )/ y ) )

def ml_error(model_name, y, yhat):
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({'Model Name': model_name,
                        'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index=[0] )

### Pré-Processamento

#### Carregando os dados

In [ ]:
# low_memory = False (ler todo o arquivo na mesma hora coloca na memoria)
df_sales_raw = pd.read_csv ('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv ('data/store.csv', low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

#### Changes

In [ ]:
df1 = df_raw.copy()

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# rename columns
df1.columns = cols_new

In [ ]:
# mudar o types da coluna date
df1['date'] = pd.to_datetime(df1['date'])

#### Tratamento de valores inconsistentes

#### Tratamento de valores faltantes

In [ ]:
#competition_distance - distancia em metros da loja competidora proxima
    # uma das formas de eliminar os NAs é preenchendo as linhas faltantes
    # utilizando a lógica de que talvez  NA é pq a loja competidora está bem distante
    # e para isso irei preencher um valor qualquer 200000.0 (maior q o valor maximo do meu dataframe) 
    
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

#competition_open_since_month - mes/ano que a loja competidora foi aberta
# quando é mais de uma coluna coloca axis=1
# foi preenchido de acordo com a data de registro
    
df1['competition_open_since_month']= df1.apply(lambda x: x['date'].month 
                                               if math.isnan(x['competition_open_since_month']) else 
                                               x['competition_open_since_month'], axis=1)

#competition_open_since_year  
df1['competition_open_since_year']= df1.apply(lambda x: x['date'].year 
                                               if math.isnan(x['competition_open_since_year']) else      
                                               x['competition_open_since_year'], axis=1)                       
#promo2_since_week
df1['promo2_since_week']= df1.apply(lambda x: x['date'].week 
                                               if math.isnan(x['promo2_since_week']) else      
                                               x['promo2_since_week'], axis=1)  
#promo2_since_year   
df1['promo2_since_year']= df1.apply(lambda x: x['date'].year 
                                               if math.isnan(x['promo2_since_year']) else      
                                               x['promo2_since_year'], axis=1)


df1['promo_interval'] = df1['promo_interval'].fillna(0)

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

#### Feature Engineering

In [ ]:
df2 = df1.copy()

In [ ]:
## extraindo novas features da coluna date
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

## features de competition
# competition since - juntas o mes - ano
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year= x['competition_open_since_year'], month= x['competition_open_since_month'] , day=1), axis=1)
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

## feature de promo
# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
# transformando em datetime
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

## feature assortment
df2['assortment']= df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x:'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day') 


# -----------------------------------------------------------------------------------------------------

#promo_interval - intervalos consecutivos quando a promo2 foi iniciada
# dicionario indicando o numero dos meses
month_map = {1: 'Jan',
            2: 'Feb',
            3: 'Mar',
            4: 'Apr',
            5: 'May',
            6: 'Jun',
            7: 'Jul',
            8: 'Aug',
            9: 'Sept',
            10: 'Oct',
            11: 'Nov',
            12: 'Dec'}

    # assumption: criadno uma coluna onde indica o mes da coluna date     
df2['month_map'] = df2['date'].dt.month.map(month_map)

    # assumption: se o mes estiver presente na coluna 'promo_interval' logo tem promocao ativa naquela data 
df2['is_promo'] = df2[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval']==0 else 
                                                            1 if x['month_map'] in x['promo_interval'].split(',') 
                                                            else 0, axis=1)

#### Seleção e Filtragem do Dataset

In [ ]:
df3 = df2.copy()

In [ ]:
# as restrições observadas para o negócio foram loja fechada e vendas nulas
df3 = df3[(df3['open'] != 0) & (df3['sales'] != 0 )]

In [ ]:
# a coluna customers é uma restrição para o modelo, pois tem essa informação daqui 6 semanas
# a não ser que eu faça outro modelo de previsão para calcular a coluna customers daqui 6 semanas.
# então por isso, foi retirar essa coluna do meu dataset para fazer a predição de vendas daqui 6 semanas.
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1)

#### Divisão entre previsores e classes / Treinamento e Teste

In [ ]:
df4 = df3.copy()

In [ ]:
# pegar as ultimas 6 semanas e fazer como teste
# data antes das ultimas 6 semanas será os meus dados de treino

# para descobrir a data antes das 6 semanas
#df4[['store', 'date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta( days=6*7 ) # resultado 2015-06-19

# training dataset
X_train = df4[df4['date'] < '2015-06-19']
y_train = X_train['sales']

# test dataset
X_test = df4[df4['date'] >= '2015-06-19']
y_test = X_test['sales']

print('Training Min Date: {}'.format(X_train['date'].min()))
print('Training Max Date: {}'.format(X_train['date'].max()))

print('\nTest Min Date: {}'.format(X_test['date'].min()))
print('Test Min Date: {}'.format(X_test['date'].max()))


# aqui, eu separei o meu dataset entre o de treino (data menor que 2015-06-19)
# e dataset de teste (data maior e igual que 2015-06-19)

#### Data preparation - Treinamento

In [ ]:
# Primeira motificação: mudar os dados categoricos para numericos
# Segunda modificação: em relação ao range, colocar todos na mesma escala.

**Normalização**

In [ ]:
# buscar uma variavel com distribuição normal
# como vimos no Numerical Variable não há uma variavel com distribuição normal
# optamos em não normalizar nenhuma variavel, pois pode afetar o meu algoritmo forçando essa normalização. 

**Rescaling**

In [ ]:
# não muda a natureza da variavel
# min-max scaler é sensivel aos outliers
# robust scaler

In [ ]:
rs = RobustScaler()
mms = MinMaxScaler()

# competition distance
X_train['competition_distance'] = rs.fit_transform( X_train[['competition_distance']].values )
pickle.dump(rs, open('parameter/competition_distance_scaler.pkl', 'wb'))

# competition time month
X_train['competition_time_month'] = rs.fit_transform( X_train[['competition_time_month']].values )
pickle.dump(rs, open('parameter/competition_time_month_scaler.pkl', 'wb'))

# promo time week
X_train['promo_time_week'] = mms.fit_transform( X_train[['promo_time_week']].values )
pickle.dump(mms, open('parameter/promo_time_week_scaler.pkl', 'wb'))

# year
X_train['year'] = mms.fit_transform( X_train[['year']].values )
pickle.dump(mms, open('parameter/year_scaler.pkl', 'wb'))

##### Transformação

**Enconding**

In [ ]:
# enconding - muda a variavel de categorica para numerica sem mudar o conteudo de unformação

In [ ]:
# state_holiday - One Hot Encoding
X_train = pd.get_dummies(X_train, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Enconding
le = LabelEncoder()
X_train['store_type'] = le.fit_transform(X_train['store_type'])
pickle.dump(le, open('parameter/store_type_scaler.pkl', 'wb'))

# assortment - Ordinal Enconding
assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
X_train['assortment'] = X_train['assortment'].map(assortment_dict)

**Transformação da Variável Alvo**

In [ ]:
# transformação logaritma 
X_train['sales'] = np.log1p(X_train['sales'])

y_train = np.log1p(y_train)

**Transformação de Natureza**

In [ ]:
X_train.head()

In [ ]:
## natureza ciclica - seno e cosseno
# day of week
X_train['day_of_week_sin'] = X_train['day_of_week'].apply(lambda x: np.sin(x * ( 2. * np.pi/7 )))
X_train['day_of_week_cos'] = X_train['day_of_week'].apply(lambda x: np.cos(x * ( 2. * np.pi/7 )))

# month
X_train['month_sin'] = X_train['month'].apply(lambda x: np.sin(x * ( 2. * np.pi/12 )))
X_train['month_cos'] = X_train['month'].apply(lambda x: np.cos(x * ( 2. * np.pi/12 )))

# day
X_train['day_sin'] = X_train['day'].apply(lambda x: np.sin(x * ( 2. * np.pi/30 )))
X_train['day_cos'] = X_train['day'].apply(lambda x: np.cos(x * ( 2. * np.pi/30 )))

# week of year
X_train['week_of_year_sin'] = X_train['week_of_year'].apply(lambda x: np.sin(x * ( 2. * np.pi/52 )))
X_train['week_of_year_cos'] = X_train['week_of_year'].apply(lambda x: np.cos(x * ( 2. * np.pi/52 )))

In [ ]:
# drop a coluna original que deriva a coluna_sin e coluna_cos
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
X_train = X_train.drop(cols_drop, axis=1)

In [ ]:
# SALVANDO 
with open('parameter/train_prepare.pkl', mode = 'wb') as f:
    pickle.dump([X_train, y_train], f)

#### Data preparation - Teste

In [ ]:
# Primeira motificação: mudar os dados categoricos para numericos
# Segunda modificação: em relação ao range, colocar todos na mesma escala.

# **Normalização**

# buscar uma variavel com distribuição normal
# como vimos no Numerical Variable não há uma variavel com distribuição normal
# optamos em não normalizar nenhuma variavel, pois pode afetar o meu algoritmo forçando essa normalização. 

# **Rescaling**

# não muda a natureza da variavel
# min-max scaler é sensivel aos outliers
# robust scaler

rs = RobustScaler()
mms = MinMaxScaler()

# competition distance
X_test['competition_distance'] = rs.fit_transform( X_test[['competition_distance']].values )
# pickle.dump(rs, open('/home/caroline/repos/DataScience_em_Producao/parameter/competition_distance_scaler.pkl', 'wb'))

# competition time month
X_test['competition_time_month'] = rs.fit_transform( X_test[['competition_time_month']].values )
# pickle.dump(rs, open('/home/caroline/repos/DataScience_em_Producao/parameter/competition_time_month_scaler.pkl', 'wb'))

# promo time week
X_test['promo_time_week'] = mms.fit_transform( X_test[['promo_time_week']].values )
# pickle.dump(mms, open('/home/caroline/repos/DataScience_em_Producao/parameter/promo_time_week_scaler.pkl', 'wb'))

# year
X_test['year'] = mms.fit_transform( X_test[['year']].values )
# pickle.dump(mms, open('/home/caroline/repos/DataScience_em_Producao/parameter/year_scaler.pkl', 'wb'))

##### Transformação

# **Enconding**

# enconding - muda a variavel de categorica para numerica sem mudar o conteudo de unformação

# state_holiday - One Hot Encoding
X_test = pd.get_dummies(X_test, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Enconding
le = LabelEncoder()
X_test['store_type'] = le.fit_transform(X_test['store_type'])
# pickle.dump(le, open('/home/caroline/repos/DataScience_em_Producao/parameter/store_type_scaler.pkl', 'wb'))

# assortment - Ordinal Enconding
assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
X_test['assortment'] = X_test['assortment'].map(assortment_dict)

# **Transformação da Variável Alvo**

# transformação logaritma 
X_test['sales'] = np.log1p(X_test['sales'])

y_test = np.log1p(y_test)


# **Transformação de Natureza**

## natureza ciclica - seno e cosseno
# day of week
X_test['day_of_week_sin'] = X_test['day_of_week'].apply(lambda x: np.sin(x * ( 2. * np.pi/7 )))
X_test['day_of_week_cos'] = X_test['day_of_week'].apply(lambda x: np.cos(x * ( 2. * np.pi/7 )))

# month
X_test['month_sin'] = X_test['month'].apply(lambda x: np.sin(x * ( 2. * np.pi/12 )))
X_test['month_cos'] = X_test['month'].apply(lambda x: np.cos(x * ( 2. * np.pi/12 )))

# day
X_test['day_sin'] = X_test['day'].apply(lambda x: np.sin(x * ( 2. * np.pi/30 )))
X_test['day_cos'] = X_test['day'].apply(lambda x: np.cos(x * ( 2. * np.pi/30 )))

# week of year
X_test['week_of_year_sin'] = X_test['week_of_year'].apply(lambda x: np.sin(x * ( 2. * np.pi/52 )))
X_test['week_of_year_cos'] = X_test['week_of_year'].apply(lambda x: np.cos(x * ( 2. * np.pi/52 )))



In [ ]:
# drop a coluna original que deriva a coluna_sin e coluna_cos
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
X_test = X_test.drop(cols_drop, axis=1)

In [ ]:
# SALVANDO

with open('parameter/test_prepare.pkl', mode = 'wb') as f:
    pickle.dump([X_test, y_test], f)

#### Seleção de variáveis através do Boruta

In [ ]:
# with open('parameter/train_prepare.pkl', 'rb') as f:
#     X_train, y_train = pickle.load(f)

In [ ]:
# # training and test dataset for boruta
# x_train_n = X_train.drop(['date', 'sales'], axis=1).values
# y_train_n = y_train.values.ravel()

# # define RadomForestRegressor
# rf = RandomForestRegressor( n_jobs= -1 ) # warm_start = True,

# # define boruta
# boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( x_train_n, y_train_n )

In [ ]:
# cols_selected = boruta.support_.tolist()

# # best features
# x_train_fs = X_train.drop(['date', 'sales'], axis=1)
# cols_selected_boruta = x_train_fs.iloc[:, cols_selected].columns.to_list()


# # not selected boruta
# cols_not_selected_boruta = list(np.setdiff1d(x_train_fs.columns, cols_selected_boruta))

In [ ]:
# cols_selected_boruta

In [ ]:
cols_selected_boruta = ['store',
 'promo',
 'store_type',
 'assortment',
 'competition_distance',
 'competition_open_since_month',
 'competition_open_since_year',
 'promo2',
 'promo2_since_week',
 'promo2_since_year',
 'competition_time_month',
 'promo_time_week',
 'day_of_week_sin',
 'day_of_week_cos',
 'month_sin',
 'month_cos',
 'day_sin',
 'day_cos',
 'week_of_year_sin',
 'week_of_year_cos']

# columns to add
feat_to_add = ['date', 'sales']

# final feature
cols_selected_boruta_full = cols_selected_boruta.copy()
cols_selected_boruta_full.extend( feat_to_add )

### MACHINE LEARNING MODELLING

In [ ]:
with open('parameter/train_prepare.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)
    
with open('parameter/test_prepare.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

In [ ]:
X_train.head()

In [ ]:
x_train = X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

# Tiem Series Data Preparation
x_training = X_train[cols_selected_boruta_full]

#### 7.1 Average Model

In [ ]:
aux1 = x_test.copy()
aux1['sales'] = y_test.copy()

# prediction
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
aux1 = pd.merge(aux1, aux2, how='left', on='store')
yhat_baseline = aux1['predictions']

# performance
baseline_result = ml_error('Average Model', np.expm1( y_test ), np.expm1( yhat_baseline ) )
baseline_result

#### 7.2 Linear Regression Model

In [ ]:
# model
lr = LinearRegression().fit(x_train, y_train)

# prediction
yhat_lr = lr.predict(x_test)

# performance
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

##### 7.2.1 Linear Regression Model - Cross Validation

In [ ]:
x_training

In [ ]:
lr_result_cv = cross_validation( x_training, 5, 'Linear Regression', lr, verbose=False)
lr_result_cv

#### 7.3 Linear Regression Regularized Model - Lasso

In [ ]:
# model
lrr = Lasso(alpha=0.01).fit(x_train, y_train)

# prediction
yhat_lrr = lrr.predict(x_test)

# performance
lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result

##### 7.3.1 Lasso - Cross Validation

In [ ]:
lrr_result_cv = cross_validation( x_training, 5, 'Lasso', lrr, verbose=False)
lrr_result_cv

#### 7.4 Random Forest Regressor

In [ ]:
# model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(x_train, y_train)

# prediction
yhat_rf = rf.predict(x_test)

# performance
rf_result = ml_error('Random Forest Regression', np.expm1(y_test), np.expm1(yhat_rf))
rf_result

##### 7.4.1 Random Forest Regressor- Cross Validation

In [ ]:
rf_result_cv = cross_validation( x_training, 5, 'Random Forest Regression', rf, verbose=True)
rf_result_cv

#### 7.5 XGBoost Regressor

In [ ]:
# model
model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                             n_estimators=1000, 
                             eta=0.01,
                             max_depth=10,
                             subsample=0.7,
                             colsample_bytee=0.9).fit(x_train, y_train)

# prediction
yhat_xgb = model_xgb.predict(x_test)

# performance
xgb_result = ml_error('XGBoost Regression', np.expm1(y_test), np.expm1(yhat_xgb))
xgb_result

##### 7.5.1 XGBoots Regressor - Cross Validation

In [ ]:
xgb_result_cv = cross_validation( x_training, 5, 'XGBoots', model_xgb, verbose=True)
xgb_result_cv

#### 7.5 Compare Model's Performance

In [ ]:
modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result])
modelling_result.sort_values('RMSE')

##### 7.6.2 Real Performace - Cross Validation

In [ ]:
modelling_result = pd.concat([lr_result_cv, lrr_result_cv, rf_result_cv, xgb_result_cv])
modelling_result.sort_values('RMSE CV')

### Fine Tunning

**Implementação do Random Search do modelo XGBoost**

#### 8.1 Random Search

In [ ]:
param = {
        'n_estimators': [1500, 1700, 2500, 3000, 3500],
        'eta': [0.01, 0.03],
        'max_depth': [3, 5, 9],
        'subsample': [0.1, 0.5, 0.7],
        'colsample_bytree': [0.3, 0.7, 0.9],
        'min_child_weight': [3, 8, 15] }

MAX_EVAL = 5

In [ ]:
final_result = pd.DataFrame()

for i in range( MAX_EVAL ):

    # choose values for parameters randomly

    hp = { k: random.sample( v, 1 )[0] for k, v in param.items() }

    print( hp )


    # model

    model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                                n_estimators=hp['n_estimators'],
                                eta=hp['eta'],
                                max_depth=hp['max_depth'],
                                subsample=hp['subsample'],
                                colsample_bytee=hp['colsample_bytree'],
                                min_child_weight=hp['min_child_weight'] )

    # performance
    result = cross_validation( x_training, 5, 'XGBoost Regressor', model_xgb, verbose=True )

    final_result = pd.concat( [final_result, result] )

final_result

#### 8.2 Final Model

In [ ]:
param_tuned = {
          'n_estimators': 3500,
          'eta': 0.03,
          'max_depth': 9,
          'subsample': 0.1,
          'colsample_bytree': 0.7,
          'min_child_weight': 3
}

# {'n_estimators': 3500, 'eta': 0.03, 'max_depth': 9, 'subsample': 0.1, 'colsample_bytree': 0.7, 'min_child_weight': 3}

In [ ]:
# model
model_xgb_tuned = xgb.XGBRegressor( objective='reg:squarederror',
                                    n_estimators=param_tuned['n_estimators'],
                                    eta=param_tuned['eta'],
                                    max_depth=param_tuned['max_depth'],
                                    subsample=param_tuned['subsample'],
                                    colsample_bytee=param_tuned['colsample_bytree'],
                                    min_child_weight=param_tuned['min_child_weight'] ).fit( x_train, y_train )

# prediction
yhat_xgb_tuned = model_xgb_tuned.predict( x_test )

# performance
xgb_result_tuned = ml_error( 'XGBoost Regressor', np.expm1( y_test ), np.expm1(yhat_xgb_tuned ) )
xgb_result_tuned

In [ ]:
# Para salvar o modelo, que vai levar + de 1 dia.

pickle.dump(model_xgb_tuned, open('model/model_rossmann.pkl', 'wb'))

In [ ]:
with open('model/model_rossmann.pkl', 'rb') as f:
    model_xgb_tuned = pickle.load(f)

In [ ]:
# prediction
yhat_xgb_tuned = model_xgb_tuned.predict( x_test )

### Interpretação do ERRO

In [ ]:
df9 = X_test[cols_selected_boruta_full]

In [ ]:
# rescale
df9['sales'] = np.expm1(df9['sales'])
df9['predictions'] = np.expm1(yhat_xgb_tuned)

#### Business Performance

In [ ]:
# sum of prediction
df91 = df9[['store', 'predictions']].groupby('store').sum().reset_index()

# MAE and MAPE
# parametro sale com predictions
df9_aux1 = df9[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_error(x['sales'], x['predictions'])).reset_index().rename(columns={0 : 'MAE'})

df9_aux2 = df9[['store', 'sales', 'predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error(x['sales'], x['predictions'])).reset_index().rename(columns={0 : 'MAPE'})

# merge
df9_aux3 = pd.merge(df9_aux1, df9_aux2, how='inner', on='store')
df92 = pd.merge(df91, df9_aux3, how='inner', on='store')

# Scenarios
df92['worst_scenario'] = df92['predictions'] - df92['MAE']
df92['best_scenario'] = df92['predictions'] + df92['MAE']

# Order columns
df92 = df92[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']]

In [ ]:
df92.sort_values(by='MAPE',ascending=True ).head()

In [ ]:
# como observado, existe loja que tem a porcentagem de error maior que 50%, logo fica dificil obter a sua predição de vendas de 6 semanas.
sns.scatterplot(x='store', y='MAPE', data=df92);

#### Total Performace

In [ ]:
# mostrar a soma de todos os cenários de todas as lojas
df93 = df92[['predictions', 'worst_scenario', 'best_scenario']].apply(lambda x: np.sum(x), axis=0).reset_index().rename(columns={'index': 'scenario', 0: 'Values'})
df93['Values'] = df93['Values'].map('R${:,.2f}'.format)
df93

### Deploy Model to Production

In [ ]:
# Para salvar o modelo, que vai levar + de 1 dia.

pickle.dump(model_xgb_tuned, open('/home/caroline/repos/DataScience_em_Producao/model/model_rossmann.pkl', 'wb'))

#### Rossamann Class

In [ ]:
import pickle
import inflection
import pandas as pd
import numpy as np
import math
import datetime


class Rossmann(object):
    def __init__(self):
        
        self.home_path='/home/caroline/repos/rossmann_sales_project/'
        
        self.competition_distance_scaler = pickle.load(
                 open(self.home_path + 'parameter/competition_distance_scaler.pkl', 'rb'))
        
        self.competition_time_month_scaler = pickle.load(
                open(self.home_path + 'parameter/competition_time_month_scaler.pkl', 'rb'))
        
        self.promo_time_week_scaler = pickle.load(
                open(self.home_path + 'parameter/promo_time_week_scaler.pkl', 'rb'))
        
        self.year_scaler = pickle.load(
                open(self.home_path + 'parameter/year_scaler.pkl', 'rb'))
        
        self.store_type_scaler = pickle.load(
                open(self.home_path + 'parameter/store_type_scaler.pkl', 'rb'))
    
    def data_cleaning(self, df1):

        ## 1.1 Rename Columns

        cols_old = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo',
               'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
               'CompetitionDistance', 'CompetitionOpenSinceMonth',
               'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
               'Promo2SinceYear', 'PromoInterval']

        snakecase = lambda x: inflection.underscore(x)

        cols_new = list(map(snakecase, cols_old))

        # rename columns
        df1.columns = cols_new


        ## 1.3 Data Types

        # mudar o types da coluna date
        df1['date'] = pd.to_datetime(df1['date'])

        ## 1.5 Fillout NA

        #competition_distance - distancia em metros da loja competidora proxima
            # uma das formas de eliminar os NAs é preenchendo as linhas faltantes
            # utilizando a lógica de que talvez  NA é pq a loja competidora está bem distante
            # e para isso irei preencher um valor qualquer 200000.0 (maior q o valor maximo do meu dataframe) 

        df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

        #competition_open_since_month - mes/ano que a loja competidora foi aberta
            # quando é mais de uma coluna coloca axis=1

        df1['competition_open_since_month']= df1.apply(lambda x: x['date'].month 
                                                       if math.isnan(x['competition_open_since_month']) else 
                                                       x['competition_open_since_month'], axis=1)

        #competition_open_since_year  
        df1['competition_open_since_year']= df1.apply(lambda x: x['date'].year 
                                                       if math.isnan(x['competition_open_since_year']) else      
                                                       x['competition_open_since_year'], axis=1)                       
        #promo2_since_week
        df1['promo2_since_week']= df1.apply(lambda x: x['date'].week 
                                                       if math.isnan(x['promo2_since_week']) else      
                                                       x['promo2_since_week'], axis=1)  
        #promo2_since_year   
        df1['promo2_since_year']= df1.apply(lambda x: x['date'].year 
                                                       if math.isnan(x['promo2_since_year']) else      
                                                       x['promo2_since_year'], axis=1)
        #promo_interval - intervalos consecutivos quando a promo2 foi iniciada
        # dicionario indicando o numero dos meses
        month_map = {1: 'Jan',
                    2: 'Feb',
                    3: 'Mar',
                    4: 'Apr',
                    5: 'May',
                    6: 'Jun',
                    7: 'Jul',
                    8: 'Aug',
                    9: 'Sept',
                    10: 'Oct',
                    11: 'Nov',
                    12: 'Dec'}
        df1['promo_interval'] = df1['promo_interval'].fillna(0)
            # assumption: criadno uma coluna onde indica o mes da coluna date     
        df1['month_map'] = df1['date'].dt.month.map(month_map)

            # assumption: se o mes estiver presente na coluna 'promo_interval' logo tem promocao ativa naquela data 
        df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval']==0 else 
                                                                    1 if x['month_map'] in x['promo_interval'].split(',') 
                                                                    else 0, axis=1)


        ## 1.6 Change Types

        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

        df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
        df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

        
        return df1
    

    
    def feature_engineering(self, df2):

        ## extraindo novas features da coluna date
        # year
        df2['year'] = df2['date'].dt.year

        # month
        df2['month'] = df2['date'].dt.month

        # day
        df2['day'] = df2['date'].dt.day

        # week of year
        df2['week_of_year'] = df2['date'].dt.isocalendar().week

        # year week
        df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

        ## features de competition
        # competition since - juntas o mes - ano
        df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year= x['competition_open_since_year'], 
                                                                         month= x['competition_open_since_month'] , day=1), axis=1)
        df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

        ## feature de promo
        # promo since
        df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
        # transformando em datetime
        df2['promo_since'] = df2['promo_since'].apply(
                                lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))
        df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

        ## feature assortment
        df2['assortment']= df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

        # state holiday
        df2['state_holiday'] = df2['state_holiday'].apply(lambda x:'public_holiday' if x == 'a' else 
                                                          'easter_holiday' if x == 'b' else 
                                                          'christmas' if x == 'c' else 'regular_day') 

        # 3.0 - Filtragem das Variáveis
        ## 3.1 - Filtragem das linhas

        # as restrições observadas para o negócio foram loja fechada e vendas nulas
        df2 = df2[(df2['open'] != 0)]

        ## 3.2 - Seleção

        # a coluna customers é uma restrição para o modelo, pois tem essa informação daqui 6 semanas
        # a não ser que eu faça outro modelo de previsão para calcular a coluna customers daqui 6 semanas.
        # então por isso, foi retirar essa coluna do meu dataset para fazer a predição de vendas daqui 6 semanas.
        cols_drop = [ 'open', 'promo_interval', 'month_map']
        df2 = df2.drop(cols_drop, axis=1)
        
        return df2
    
    def data_preparation(self, df5):

        ## 5.2 Rescaling
        # competition distance
        df5['competition_distance'] = self.competition_distance_scaler.transform( df5[['competition_distance']].values )


        # competition time month
        df5['competition_time_month'] = self.competition_time_month_scaler.transform( df5[['competition_time_month']].values )
        

        # promo time week
        df5['promo_time_week'] = self.promo_time_week_scaler.transform( df5[['promo_time_week']].values )
        

        # year
        df5['year'] = self.year_scaler.transform( df5[['year']].values )
        
        


        ## 5.3 Transformação

        ### 5.3.1 Enconding 

        # enconding - muda a variavel de categorica para numerica sem mudar o conteudo de unformação

        # state_holiday - One Hot Encoding
        df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

        # store_type - Label Enconding
        df5['store_type'] = self.store_type_scaler.transform(df5['store_type'])


        # assortment - Ordinal Enconding
        assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
        df5['assortment'] = df5['assortment'].map(assortment_dict)
        
        ## natureza ciclica - seno e cosseno
        # day of week
        df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x * ( 2. * np.pi/7 )))
        df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x * ( 2. * np.pi/7 )))

        # month
        df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * ( 2. * np.pi/12 )))
        df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * ( 2. * np.pi/12 )))

        # day
        df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * ( 2. * np.pi/30 )))
        df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * ( 2. * np.pi/30 )))

        # week of year
        df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x * ( 2. * np.pi/52 )))
        df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x * ( 2. * np.pi/52 )))
        
        cols_selected = ['store', 'promo', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month',
                                'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 
                                'competition_time_month', 'promo_time_week', 'day_of_week_sin', 'day_of_week_cos', 
                                'month_sin', 'month_cos', 'day_sin', 'day_cos', 'week_of_year_sin', 'week_of_year_cos']
        
        return df5[cols_selected]
    
    def get_prediction(self, model, original_data, test_data):
        # prediction
        pred = model.predict(test_data)
        
        # join pred into the original data
        original_data['prediction'] = np.expm1(pred)
        
        return original_data.to_json(orient='records', date_format='iso')

#### API Handler

In [ ]:
import pickle
import pandas as pd
from flask import Flask, request, Response
from rossmann.Rossmann import Rossmann

# loading model
model = pickle.load(open('model/model_rossmann.pkl', 'rb'))

# initialize API
app = Flask(__name__)

@app.route('/rossmann/predict', methods=['POST'])

def rossmann_predict():
    test_json = request.get_json()
    
    if test_json: # there is data
        if isinstance(test_json, dict): # unique example
            test_raw = pd.DataFrame(test_json, index=[0])
            
        else: # multiple example
            test_raw = pd.DataFrame(test_json, columns= test_json[0].keys())
            
        # Instantiate Rossmann class
        pipeline = Rossmann()
        
        # data cleaning
        df1 = pipeline.data_cleaning(test_raw)
        
        # feature engineering
        df2 = pipeline.feature_engineering(df1)
        
        # data preparation
        df3 = pipeline.data_preparation(df2)
        
        # prediction
        df_response = pipeline.get_prediction(model, test_raw, df3)
        
        return df_response
    
    else: 
        return Response('{}', status=200, mimetype='application/json')
    

if __name__ == '__main__':
    app.run('0.0.0.0')

#### API Tester

In [ ]:
import requests

In [ ]:
# loading test dataset
df10 = pd.read_csv('/home/caroline/repos/DataScience_em_Producao/data/test.csv')

In [ ]:
# merge test dataset + store
df_test = pd.merge(df10, df_store_raw, how='left', on = 'Store')

# choose store for prediction
df_test = df_test[df_test['Store'].isin([58,50,23])]

# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop('Id', axis = 1)



In [ ]:
# convert dataframe to json
data = json.dumps(df_test.to_dict(orient='records'))

In [ ]:
# API call
# url = 'http://0.0.0.0:5000/rossmann/predict' #/rossman/predict é o endpoint em ambiente local
url = 'https://teste-deploy-render-bc3n.onrender.com/rossmann/predict'

header = {'Content-type': 'application/json'} # indica qual tipo de dado esta recebendo
data = data

r = requests.post( url, data = data, headers = header)
# metodo POST serve para enviar o dado

print(f'Status code {r.status_code}')
# para indicar se a request é válida
# reorno de 200 significa que está tudo okay

In [ ]:
d1 = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [ ]:
d2 = d1[['store', 'prediction']].groupby('store').sum().reset_index()

for i in range(len(d2)):
    print('Store Number {} will sell R${:,.2f} in the next 6 weeks'.format(
                        d2.loc[i, 'store'],
                        d2.loc[i, 'prediction'] ) )

In [ ]:
d_plot = d1[['store', 'prediction', 'year_week']].groupby(['store', 'year_week']).sum().reset_index()

In [ ]:
d_plot.head()

In [ ]:
# f, ax = plt.subplots()
f = plt.figure(figsize=(20,8))
ax = sns.lineplot(data= d_plot, x= 'year_week', y= 'prediction', hue='store', marker='o')

# for v in d_plot.iterrows():
#     ax.text(v[1][0] , v[1][2], f'{round(v[1][2]/1000000, 2)} M', 
#             horizontalalignment='left', size='medium', color='black', weight='semibold')
# ax.ticklabel_format(style='plain', axis="y")